Tests a dog vs. cat classifier with dropout and without dropout

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


Load the dataset and apply data augmentation

In [2]:
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder(root='./data', transform=transform)

# Split the dataset into train and test sets
num_samples = len(dataset)
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)
test_size = num_samples - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# verify
# for file in dataset.imgs:
#     print(file)
    
    
# Define the dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Define the model, include forward dropout option

In [3]:
# Define the model
class ConvNet(nn.Module):
    def __init__(self, dropout=False):
        super(ConvNet, self).__init__()
        self.dropout = dropout
        
        # what are all the nums for? how do i set them appropriately?
        self.conv1 = nn.Conv2d(3, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 3)
        
        # 153664 is from the error message (replace with dif # and see)
        self.fc1 = nn.Linear(153664, 128)
        self.fc2 = nn.Linear(128, 2)
        if self.dropout:
            self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        batch_size, output_channels, height, width = x.shape
        input_features = output_channels * height * width
        x = x.view(-1, input_features)  # flatten the input tensor
        x = torch.relu(self.fc1(x))
        if self.dropout:
            x = self.dropout(x)
        x = self.fc2(x)
        return x

Train & use the model

In [4]:
# Train the model without dropout
model_without_dropout = ConvNet(dropout=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_without_dropout.parameters())

train_losses_without_dropout = []
val_losses_without_dropout = []

num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model_without_dropout.train()
    train_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_without_dropout(images)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss = train_loss / len(train_dataloader.dataset)
    train_losses_without_dropout.append(train_loss)
    
    # Validation
    model_without_dropout.eval()
    val_loss = 0.0
    
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_without_dropout(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)
            
    val_loss = val_loss / len(val_dataloader.dataset)
    val_losses_without_dropout.append(val_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.3f}, Validation Loss: {val_loss:.3f}')

Epoch 1/10, Training Loss: 3.817, Validation Loss: 4.932
Epoch 2/10, Training Loss: 2.323, Validation Loss: 1.748
Epoch 3/10, Training Loss: 1.018, Validation Loss: 0.726
Epoch 4/10, Training Loss: 0.652, Validation Loss: 0.689
Epoch 5/10, Training Loss: 0.599, Validation Loss: 0.627
Epoch 6/10, Training Loss: 0.554, Validation Loss: 0.638
Epoch 7/10, Training Loss: 0.510, Validation Loss: 0.644
Epoch 8/10, Training Loss: 0.486, Validation Loss: 0.640
Epoch 9/10, Training Loss: 0.448, Validation Loss: 0.683
Epoch 10/10, Training Loss: 0.410, Validation Loss: 0.685


In [5]:
model_without_dropout.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_without_dropout(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy without Dropout: {100 * correct / total}')

Test Accuracy without Dropout: 62.33766233766234


In [6]:
# Train the model with dropout
model_with_dropout = ConvNet(dropout=True).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_with_dropout.parameters())

num_epochs = 10

train_losses_with_dropout = []
val_losses_with_dropout = []

for epoch in range(num_epochs):
    # Training
    model_with_dropout.train()
    train_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_with_dropout(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss = train_loss / len(train_dataloader.dataset)
    train_losses_with_dropout.append(train_loss)
    
    # Validation
    model_with_dropout.eval()
    val_loss = 0.0
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_dropout(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)
    val_loss = val_loss / len(val_dataloader.dataset)
    val_losses_with_dropout.append(val_loss)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.3f}, Validation Loss: {val_loss:.3f}')

Epoch 1/10, Training Loss: 6.727, Validation Loss: 5.362
Epoch 2/10, Training Loss: 2.290, Validation Loss: 1.304
Epoch 3/10, Training Loss: 0.882, Validation Loss: 0.876
Epoch 4/10, Training Loss: 0.675, Validation Loss: 0.683
Epoch 5/10, Training Loss: 0.641, Validation Loss: 0.665
Epoch 6/10, Training Loss: 0.549, Validation Loss: 0.688
Epoch 7/10, Training Loss: 0.512, Validation Loss: 0.693
Epoch 8/10, Training Loss: 0.479, Validation Loss: 0.731
Epoch 9/10, Training Loss: 0.469, Validation Loss: 0.744
Epoch 10/10, Training Loss: 0.430, Validation Loss: 0.797


In [7]:
# Test the model
model_with_dropout.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_with_dropout(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy with Dropout: {100 * correct / total}')

Test Accuracy with Dropout: 64.93506493506493
